# Assignment 5. A function to compute the flow after a deflection angle

Write a function, say `OSWFlow(U1,theta,T1, p1, theta)`, that gives for the flow 1 entering a deflection angle $\theta$, the characteristics of the flow after the OSW.  That is:

- Is there an OSW?
- $\text{Ma}_1$ and $\text{Ma}_2$
- Angle of the OSW ($\beta$)
- Flow conditions after the OSW: $u_2$, $T_2$ and $p_2$

Don't forget to properly document the function.

---------------------------------
# A5: GROUP 1 SOLUTION (Pol Padilla, Ferran de Miguel, Alejandro Sanchez)
---------------------------------

Normal shock waves are a particular case of the more general [oblique shock waves](https://en.wikipedia.org/wiki/Oblique_shock). In a oblique shock, supersonic flow is deviated by an obstacle. We are going to consider, for the shake of simplicity that the obstacle is a corner of angle $\theta$, and the flow is everywhere uniform and parallel to ground

![image.png](OSW.png)



## 0. Secondary support functions:

In [66]:
from IPython.display import Latex
import numpy as np
import matplotlib.pyplot as plt

def print_separator():
    print("--- --- --- --- --- --- --- --- --- --- --- ---")

Basic compressible flow functions about the fluid:

In [67]:
def gasConstant(M=0.0289647):
    """
    Returns the gas constant for a gas with molecular mass M (in J/molK)
    
    """
    R = 8.314463 # Universal Gas Constant
    return R/M

def sound_velocity(gamma=1.4,M=0.0289647,T=293.15):
    """
    Returns the velocity of sound for a gas with adiabatic index gamma and molecular mass M,
    at the temperature T (in K).
    The default arguments are for air at T = 20 C deg = 293 K
    
    Usage:
    
    c = sound_velocity(gamma = 1.34, M = 0.045, T = 312)
    """
    from numpy import sqrt
    r = gasConstant(M=M)
    return sqrt(gamma*r*T)

def MaNumber(u,gamma=1.4,M=0.0289647,T=293.15):
    """
    Returns the Mach number for a velocity u for a gas with adiabatic index gamma and molecular mass M,
    at the temperature T (in K).
    The default arguments are for air at T = 20 C deg = 293 K
    
    Usage:
    
    Ma = MaNumber(u = 750, gamma = 1.34, M = 0.045, T = 312)
    """

    return u/sound_velocity(gamma,M,T)



Normal shock wave functions:

In [68]:
def MaStar(Ma,gamma=1.4):
    """ Returns Ma* as function of Ma """
    return np.sqrt((gamma+1)/(2/Ma**2+gamma-1))

def MaShockwave(Ma,gamma=1.4):
    """
    Return the Mach number after a normal shock wave as funtion of Ma before the shock wave.
    """
    if np.any(Ma) < 1:
        print('Ma has to be greater or equal to 1')
        return
    else:
        Ma2 = Ma**2
        beta = (gamma-1)/2
        Ma22 = (1+beta*Ma2)/(gamma*Ma2-beta)
        return np.sqrt(Ma22)
    
def rho2_rho1(Ma,gamma=1.4):
    """Returns rho_2/rho_1 for a shock wave with Mach number Ma > 1"""
    if np.any(Ma) < 1:
        print('Ma has to be greater or equal to 1')
        return
    else:
        return MaStar(Ma,gamma=gamma)**2
    
def u2_u1(Ma,gamma=1.4):
    """Returns u_2/u_1 for a shock wave with Mach number Ma > 1"""
    if np.any(Ma) < 1:
        print('Ma has to be greater or equal to 1')
        return
    else:
        return 1/MaStar(Ma,gamma=gamma)**2

def p2_p1(Ma,gamma=1.4):
    """Returns p_2/p_1 for a shock wave with Mach number Ma > 1"""
    if np.any(Ma) < 1:
        print('Ma has to be greater or equal to 1')
        return
    else:
        return 1 + 2*gamma/(gamma+1)*(Ma**2-1)
    
def T2_T1(Ma,gamma=1.4):
    """Returns T_2/T_1 for a shock wave with Mach number Ma > 1"""
    if np.any(Ma) < 1:
        print('Ma has to be greater or equal to 1')
        return
    else:
        return p2_p1(Ma,gamma)/rho2_rho1(Ma,gamma)

OSW secondary functions:

In [69]:
def ThetaOSW(Ma,beta,gamma=1.4):
    """
    Returns the deflection angle theta for an oblique shock wave, given the incident Ma number and the angle os the SW, beta. 
    """
    tanTheta = 2/np.tan(beta)*(Ma**2*np.sin(beta)**2-1)/(Ma**2*(gamma+np.cos(2*beta))+2)
    return np.arctan(tanTheta)

def MaxThetaOSW(Ma,gamma=1.4):
    """
    Returns the beta OSW angle corresponding to the maximum theta value (in degrees) for a given Mach 1.
    """
    from scipy.optimize import brent
    def f(beta):
        return -ThetaOSW(Ma,beta,gamma)
    interval = (np.deg2rad(50),np.deg2rad(80))
    betaMax = brent(f,brack=interval)
    return np.rad2deg(betaMax),np.rad2deg(ThetaOSW(Ma,betaMax,gamma))

from scipy.optimize import brentq
def betaOSW(Ma,theta,gamma=1.4):
    """Returns the 2 beta values for oblique shock wave:

    Args:
        Ma (float): Mach number
        theta (float): in degrees
        gamma (float): Defaults to 1.4.

    Returns:
        beta values, in degrees
        a boolean if there is or not a solution
    """
    
    theta = np.deg2rad(theta)
    MaxBeta,MaxTheta = MaxThetaOSW(Ma)
    MaxBeta = np.deg2rad(MaxBeta)
    MaxTheta = np.deg2rad(MaxTheta)
    
    if theta > MaxTheta:
        print("No solution for the oblique shock wave (theta > MaxTheta); MaxTheta = {0:3g}".format(np.rad2deg(MaxTheta)))
        return None,None,False
    
    def func(beta):
        return 2/np.tan(beta)*(Ma**2*np.sin(beta)**2-1)/(Ma**2*(gamma+np.cos(2*beta))+2)-np.tan(theta)
    
    betaWeak = brentq(func,0.001,MaxBeta)
    betaStrong = brentq(func,MaxBeta,np.pi/2)
    
    return np.rad2deg(betaWeak),np.rad2deg(betaStrong), True
    
    

## 1. MAIN FUNCTION OSWFLOW:

In [70]:
def print_separator():
    print("--- --- --- --- --- --- --- --- --- --- --- ---")

def OSWFLOW(U1,T1,p1,theta,gamma=1.4,M=0.0289647):
    
    """Prints and returns data of the OSW:
    
    Args:
        U1 (float): flow velocity [m/s]
        T1 (float): flow temperature [K]
        p1 (float): flow pressure [Pa]
        theta (float): surface deflection angle [deg]
        gamma (float): adiabatic index; defaults to 1.4 
        M (float): molecular mass; defaults to 0.0289647 [kg/mol]

    Returns:
        OSWtrue (bool): True for a solution, False if no OSW; (if False, all other values are None)
        betaWeak (float): weak beta angle [deg]
        betaStrong (float): strong beta angle [deg] 
        Ma_2 (float): Mach after the OSW [-]
        U2 (float): flow velocity after the OSW [m/s]
        T2 (float): flow temperature after the OSW [K]
        p2 (float): flow pressure after the OSW [Pa]
        rho2 (float): flow density after the OSW [kg/m3]
    """
    
    # UNDISTURBED FLOW 1:
    r = gasConstant(M)              # [J/molK]
    rho1 = p1/(r*T1)                # [kg/m3]
    c1 = sound_velocity(gamma,M,T1) # [m/s]
    Ma = MaNumber(U1,gamma,M,T1)    # [-]
    print_separator()
    print("U1 [m/s] = {0:3f}".format(U1))
    print("T1 [K] = {0:3f}".format(T1))
    print("p1 [Pa] = {0:3f}".format(p1))
    print("rho1 [kg/m3] = {0:3f}".format(rho1))
    print("Ma1 = {0:3f}".format(Ma))
    print_separator()
    if Ma < 1: 
        print("Subsonic flow, there is no Shock Wave... ending function")
        print_separator()
        return False, None, None, None, None, None, None, None
    
    # BETA solutions for the OSW and testing if there is:
    betaWeak,betaStrong,OSWtrue = betaOSW(Ma,theta,gamma) # [deg,deg,bool]
    if OSWtrue is False:
        print("There is no OSW, the angle of the plate is too much inclinated and the shock wave has detached... ending function")
        print_separator()
        return OSWtrue, None, None, None, None, None, None, None
    
    print("The weak and strong solutions of beta respectively (deg): [{0} , {1}]".format(betaWeak,betaStrong))
    print("In nature, the weak solution is the shock wave that manifests, the solution will proceed with it...")
    
    theta = np.deg2rad(theta)   # [rad]
    beta = np.deg2rad(betaWeak) # [rad]
    
    # Computation of the normal_shock wave inside the oblique:
    Ma_n = Ma * np.sin(beta)            # [-]
    Ma_n2 = MaShockwave(Ma_n,gamma)     # [-]
    Ma_2 = Ma_n2 / np.sin(beta-theta)   # [-]
    print_separator()
    print("The resulting Mach after the OSW Ma2 = {0:3f}".format(Ma_2))
    
    # Conditions after the OSW (using the normal Mach Ma_n):
    p2p1 = p2_p1(Ma_n,gamma)  
    t2t1 = T2_T1(Ma_n,gamma)
    rho2rho1 = rho2_rho1(Ma_n,gamma)
    p2 = p2p1 * p1              # [Pa]
    T2 = t2t1 * T1              # [K]
    rho2 = rho2rho1 * rho1      # [kg/m3]
    # velocity after OSW
    c2 = sound_velocity(gamma,M,T2) # [m/s]
    U2 = c2 * Ma_2                  # [m/s]
    
    print_separator()
    print("U2 [m/s] = {0:3f}".format(U2))
    print("T2 [K] = {0:3f}".format(T2))
    print("p2 [Pa] = {0:3f}".format(p2))
    print("rho2 [kg/m3] = {0:3f}".format(rho2))
    print_separator()
    
    return OSWtrue, betaWeak, betaStrong, Ma_2, U2, T2, p2, rho2
    
    

In [71]:
### TESTING:
U1= 750 # [m/s]
T1= 298 # [K]
p1 = 1*101325 # [Pa]s
theta = 15 # [deg]
OSWtrue, betaWeak, betaStrong, Ma_2, U2, T2, p2, rho2 = OSWFLOW(U1,T1,p1,theta,gamma=1.4,M=0.0289647)

--- --- --- --- --- --- --- --- --- --- --- ---
U1 [m/s] = 750.000000
T1 [K] = 298.000000
p1 [Pa] = 101325.000000
rho1 [kg/m3] = 1.184500
Ma1 = 2.167238
--- --- --- --- --- --- --- --- --- --- --- ---
The weak and strong solutions of beta respectively (deg): [41.852515708522574 , 81.31509565567059]
In nature, the weak solution is the shock wave that manifests, the solution will proceed with it...
--- --- --- --- --- --- --- --- --- --- --- ---
The resulting Mach after the OSW Ma2 = 1.596438
--- --- --- --- --- --- --- --- --- --- --- ---
U2 [m/s] = 626.166768
T2 [K] = 382.809610
p2 [Pa] = 230289.852757
rho2 [kg/m3] = 2.095689
--- --- --- --- --- --- --- --- --- --- --- ---


In [73]:
# Try requesting help:
OSWFLOW?

Signature: OSWFLOW(U1, T1, p1, theta, gamma=1.4, M=0.0289647)
Docstring:
Prints and returns data of the OSW:

Args:
    U1 (float): flow velocity [m/s]
    T1 (float): flow temperature [K]
    p1 (float): flow pressure [Pa]
    theta (float): surface deflection angle [deg]
    gamma (float): adiabatic index; defaults to 1.4 
    M (float): molecular mass; defaults to 0.0289647 [kg/mol]

Returns:
    OSWtrue (bool): True for a solution, False if no OSW; (if False, all other values are None)
    betaWeak (float): weak beta angle [deg]
    betaStrong (float): strong beta angle [deg] 
    Ma_2 (float): Mach after the OSW [-]
    U2 (float): flow velocity after the OSW [m/s]
    T2 (float): flow temperature after the OSW [K]
    p2 (float): flow pressure after the OSW [Pa]
    rho2 (float): flow density after the OSW [kg/m3]
File:      c:\users\polpa\appdata\local\temp\ipykernel_9480\1276971092.py
Type:      function


---------------------------

In [72]:
try:
    %load_ext watermark
except:
    !pip install watermark
%watermark -v -m -iv

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.6.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 12
Architecture: 64bit

matplotlib: 3.6.2
numpy     : 1.23.4

